In [ ]:
    
    def _get_hits(self, path, treename, branches):
        """
        Creates a flat numpy recarray of hits, each with their own entry in the table
        """

        # Variables defined over the whole event
        event_variables = ["M_nHits"]
        # Variables that index the geometry
        geom_variables = ["M_volName", "M_volID"]
        default_variables = event_variables + geom_variables
        # All branches to be imported
        total_branches = default_variables + branches
        # Import the data
        event_data = root2array(path, treename=treename,\
                branches=total_branches)
        # Imported variables defiend for each hit
        hit_variables = list(set(event_data[0].dtype.names)\
                           - set(event_variables))

        # Initialize a look up table that maps from hit number to event number
        hits_to_events = np.zeros(sum(event_data["M_nHits"]))
        # Create a look up table that maps from event number the range of hits IDs
        # in that event
        event_to_hits = []
        # Store the number of hits in each event
        event_to_n_hits = event_data["M_nHits"].copy().astype(int)
        # Build the look up tables
        first_hit = 0
        for event in range(len(event_data)):
            # Record the last hit in the event
            last_hit = first_hit + event_to_n_hits[event]
            # Record the range of hit IDs
            event_to_hits.append(np.arange(first_hit,last_hit))
            # Record the event of each hit
            hits_to_events[first_hit:last_hit] = event
            # Shift to the next event
            first_hit = last_hit
        # Shift the event-to-hit list into a numpy object array
        event_to_hits = np.array(event_to_hits)
        # Ensure all indexes in hits to events are integers
        hits_to_events = hits_to_events.astype(int)
        # Save the total number of hits
        total_hits = last_hit
        
        # Flatten the volume names and IDs to flat_voldIDs
        # Create a dictionary to map from name to lower flatID
        names_to_lowers = dict(zip(self.volume_names, np.arange(6)*64))
        # Assign the flatIDs for each hit
        flat_ids = np.zeros(total_hits)
        for name, vol_id, hit in zip(np.concatenate(event_data["M_volName"]),\
                                     np.concatenate(event_data["M_volID"]),\
                                     range(total_hits)):
            flat_ids[hit] = map_names[name] + vol_id
        # Save this column and name it
        all_columns = list(flat_ids.astype(int))
        all_names = ["flat_volIDs"]  
        
        # Flatten each variable into a row of all hits and name the rows
        for hit_var in hit_variables:
            all_columns.append(np.concatenate(event_data[hit_var]))
            all_names.append(hit_var)
        
        # Assign the event-wide variables to each hit, e.g. nHits and name them
        for evt_var in event_variables:
            new_column = event_data[evt_var][hits_to_events]
            all_columns.append(new_column)
            all_names.append(evt_var)
            
        # Create a record array from these columns, use the name names as before
        hits_table = np.rec.fromarrays(all_columns, names=(all_names))
        
        # Delete the non-flat recarray
        #self.hits_data_full = event_data
        del event_data
    
        # Return 
        return hits_table, hits_to_events, event_to_hits, event_to_n_hits